In [5]:
import openai
import os
from openai import OpenAI

In [14]:
client = OpenAI(base_url="https://api.proxyapi.ru/openai/v1",
                api_key=os.environ['KEY'])

In [15]:
os.environ['KEY']

'sk-gNBJ6FK2eapniI5oFeuyATUFMxUnW8ch'

In [21]:
resp = client.chat.completions.create(
    model='gpt-4o-mini',
    max_completion_tokens=5,
    messages=[{'role': 'user', 'content': 'как приготовить лосося?'}]
)

APIStatusError: Error code: 402 - {'detail': 'Insufficient balance to run this request.'}

In [22]:
resp.choices[0].messege.content

NameError: name 'resp' is not defined